In [1]:
import fastbook
fastbook.setup_book()

In [2]:
from fastai.vision.all import *
from fastbook import *

matplotlib.rc('image', cmap='Greys')

In [ ]:
#load a dataset
path = untar_data(URLs.MNIST)
Path.BASE_PATH = path
path.ls()

(#2) [Path('testing'),Path('training')]

In [18]:
#load all train images to tensor
train_image_list = get_image_files(path/'training')
train_digits = [tensor(Image.open(o)) for o in train_image_list]
train_x = (torch.stack(train_digits).float() / 255).view(-1, 28*28)
train_x.size()

torch.Size([60000, 784])

In [147]:
#load labels
train_y = tensor([int(o.parent.name) for o in train_image_list]).view(-1, 1)

In [148]:
train_y.size()

torch.Size([60000, 1])

In [168]:
#convert to Dataloader
dset = list(zip(train_x, train_y))
dl = DataLoader(dset, batch_size=256)

In [26]:
#load a valid dataset
valid_image_list = get_image_files(path/'testing')
valid_digits = [tensor(Image.open(o)) for o in valid_image_list]
valid_x = (torch.stack(valid_digits).float() / 255).view(-1, 28*28)

In [180]:
valid_y_list = [int(o.parent.name) for o in valid_image_list]
valid_y = tensor(valid_y_list).view(-1, 1)

In [181]:
valid_dset = list(zip(valid_x, valid_y))
valid_dl = DataLoader(valid_dset, batch_size=256)

In [177]:
#model
simple_net = nn.Sequential(
    nn.Linear(28*28, 30),
    nn.ReLU(),
    nn.Linear(30, 10)
)

#loss function
def mnist_loss(pred, target):
    loss = nn.CrossEntropyLoss()
    return loss(pred, target.squeeze())

#gradient calculation 
def calc_grad(x, y, model):
    pred = model(x)
    loss = mnist_loss(pred, y)
    loss.backward()

#SGD optimizer
class BasicOptim:
    def __init__(self, params, lr):
        self.params, self.lr = list(params), lr
    def step(self):
        for p in self.params: p.data -= p.grad*lr
    def zero_grad(self):
        for p in self.params: p.grad = None

#backpropagation
def train_epoch(model):
    for x, y in dl:
        calc_grad(x, y, model)
        opt.step()
        opt.zero_grad()

#accuracy
def batch_accuracy(pred, actual):
    digit_pred = pred.max(dim=1)[1]
    return (digit_pred==actual.squeeze()).float().mean()

def train_model(model, epoches):
    for i in range(epoches):
        train_epoch(model)
        print(batch_accuracy(model(valid_x), valid_y), end=' ')

In [178]:
#set parametrs
lr = 0.001
opt = BasicOptim(simple_net.parameters(), lr)
train_model(simple_net, 20)

tensor(0.1224) tensor(0.1306) tensor(0.1967) tensor(0.2845) tensor(0.3685) tensor(0.4417) tensor(0.5029) tensor(0.5436) tensor(0.5718) tensor(0.5954) tensor(0.6134) tensor(0.6297) tensor(0.6471) tensor(0.6592) tensor(0.6750) tensor(0.6856) tensor(0.6962) tensor(0.7062) tensor(0.7162) tensor(0.7238) 

In [185]:
#resnet model from fastai
dls = ImageDataLoaders.from_folder(path, train='training',valid='testing')
learn = cnn_learner(dls, resnet18, pretrained=False,
                    loss_func=F.cross_entropy, metrics=accuracy, n_out=10)
learn.fit_one_cycle(1, 0.1)

epoch,train_loss,valid_loss,accuracy,time
0,0.096962,0.042440,0.986500,09:33
